<a href="https://colab.research.google.com/github/Reine2001/Federated-learning-based_Chatbot/blob/main/Elasticsearch_Recherche_Infos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installation des dépendances nécessaires**

In [ ]:
!pip install tensorflow tensorflow-federated transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489

In [ ]:
!pip install requests elasticsearch

In [ ]:
!pip install streamlit langchain pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.6 MB/s eta 0:00:00
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 17.0 MB/s eta 0:00:00
Using cached packaging-24.1-py3-none-any.whl (53 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.1 MB/s eta 0:00:00
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Import des bibliothèques**

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
import numpy as np
import requests
import json
from transformers import DistilBertTokenizer, TFDistilBertModel


**Chargement des données**

In [ ]:
#Vérifier le chemin à utiliser
!ls /content

Droits_humains.xlsx  Police_Judiciaire.xlsx  Police_Nationale.xlsx  sample_data  Securite.xlsx


In [ ]:

#Chargement des fichiers pour les Droits_humains
Droits_humains=pd.read_excel("/content/Droits_humains.xlsx")

#Chargement des fichiers pour la Police_Judiciaire
Police_Judiciaire=pd.read_excel("/content/Police_Judiciaire.xlsx")

#Chargement des fichiers pour la Police_Nationale
Police_Nationale=pd.read_excel("/content/Police_Nationale.xlsx")

#Chargement des fichiers pour la Securite
Securite=pd.read_excel("/content/Securite.xlsx")


In [ ]:
# Afficher les premières lignes pour vérifier le chargement
print(Droits_humains.head())

                loi  Num_Article  \
0  DECRET N2012-829  Article 1 :   
1  DECRET N2012-829  Article 2 :   
2  DECRET N2012-829  Article 3 :   
3  DECRET N2012-829  Article 4 :   
4  DECRET N2012-829  Article 5 :   

                                             Article  
0  le présent décret pris en application de l art...  
1  toute personne handicapée déclarée indigente b...  
2  toute personne handicapée et non déclarée indi...  
3  la personne handicapée titulaire de la carte d...  
4  la limite d âge des personnes handicapées cand...  


**Création des embeddings avec DistilBERT**



1.   Chargement du modèle à utiliser : distilbert-base-uncased



In [ ]:

# Initialiser le modèle DistilBERT et le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')


NameError: name 'DistilBertTokenizer' is not defined




2.   Fonction pour la création d'embeddings avec fenetre glissante pour minimiser la perte des informations au-dela de 512 tokens et moyenne des embeddings pour atténuer le problème des embeddings redondants.



In [ ]:

# Fonction pour découper un texte en morceaux avec une fenêtre glissante
def sliding_window_tokenize(text, max_len=512, stride=256):
    tokens = tokenizer(text, truncation=False, return_tensors='tf')['input_ids'][0]
    token_chunks = []

    for i in range(0, len(tokens), stride):
        chunk = tokens[i:i + max_len]
        token_chunks.append(chunk)

        if len(chunk) < max_len:
            break  # Si le morceau est plus petit que la taille maximale, on arrête

    return token_chunks


In [ ]:

# Fonction pour générer des embeddings avec la fenêtre glissante
def generate_embeddings_with_sliding_window(text_list):
    all_embeddings = []

    for text in text_list:
        token_chunks = sliding_window_tokenize(text)
        chunk_embeddings = []

        for chunk in token_chunks:
            inputs = {'input_ids': tf.expand_dims(chunk, 0)}  # Ajouter une dimension batch
            outputs = model(inputs)
            embedding = outputs.last_hidden_state[:, 0, :]  # Extraire les embeddings [CLS]
            chunk_embeddings.append(embedding.numpy())

        # Moyenne des embeddings pour l'article complet
        avg_embedding = np.mean(chunk_embeddings, axis=0)
        all_embeddings.append(avg_embedding)

    return np.array(all_embeddings)




3.   Indexation dans Elasticsearch pour optimiser la recherche d'informations



In [ ]:

# URL ngrok
ngrok_url = 'https://dc48-102-180-19-51.ngrok-free.app'


In [ ]:

# Fonction pour indexer les embeddings dans ElasticSearch
def index_embeddings_in_elasticsearch(articles, embeddings, index_name, ngrok_url):
    for i, (article, embedding) in enumerate(zip(articles, embeddings)):
        doc = {
            'article_id': f'article_{i}',
            'text': article,
            'embedding': embedding.tolist()  # Transformer en liste pour JSON
        }
        response = requests.post(f'{ngrok_url}/{index_name}/_doc/',
                                 headers={"Content-Type": "application/json"},
                                 data=json.dumps(doc))
        if response.status_code != 201:
            print(f"Erreur d'indexation pour l'article {i}: {response.text}")
        else:
            print(f"Article {i} indexé avec succès.")


In [ ]:
response = requests.delete(f'{ngrok_url}/droits_humains_embeddings')
if response.status_code == 200:
    print("Index supprimé avec succès.")
else:
    print(f"Erreur lors de la suppression de l'index : {response.text}")


Index supprimé avec succès.


In [ ]:
response = requests.delete(f'{ngrok_url}/police_judiciaire_embeddings')
if response.status_code == 200:
    print("Index supprimé avec succès.")
else:
    print(f"Erreur lors de la suppression de l'index : {response.text}")


Index supprimé avec succès.


In [ ]:
response = requests.delete(f'{ngrok_url}/securite_embeddings')
if response.status_code == 200:
    print("Index supprimé avec succès.")
else:
    print(f"Erreur lors de la suppression de l'index : {response.text}")


Index supprimé avec succès.


In [ ]:
response = requests.delete(f'{ngrok_url}/police_nationale_embeddings')
if response.status_code == 200:
    print("Index supprimé avec succès.")
else:
    print(f"Erreur lors de la suppression de l'index : {response.text}")


Index supprimé avec succès.


In [ ]:

# Fonction pour traiter les données d'une institution
def process_institution(df, institution_name, ngrok_url):
    articles = []

    for index, row in df.iterrows():
        # Formater le texte de l'article avec le numéro de l'article et la loi
        formatted_text = f"Article {row['Num_Article']} Loi {row['loi']} : {row['Article']}"
        articles.append(formatted_text)

    # Générer les embeddings pour chaque article
    embeddings = generate_embeddings_with_sliding_window(articles)

    # Indexer les articles formatés et leurs embeddings dans Elasticsearch
    index_embeddings_in_elasticsearch(articles, embeddings, f"{institution_name.lower()}_embeddings", ngrok_url)



Traitement avec les fonctions ci-dessus définis

In [ ]:

# Traiter les données pour chaque institution
process_institution(Droits_humains, 'Droits_humains', ngrok_url)
process_institution(Police_Judiciaire, 'Police_Judiciaire', ngrok_url)
process_institution(Police_Nationale, 'Police_Nationale', ngrok_url)
process_institution(Securite, 'Securite', ngrok_url)

Article 0 indexé avec succès.
Article 1 indexé avec succès.
Article 2 indexé avec succès.
Article 3 indexé avec succès.
Article 4 indexé avec succès.
Article 5 indexé avec succès.
Article 6 indexé avec succès.
Article 7 indexé avec succès.
Article 8 indexé avec succès.
Article 9 indexé avec succès.
Article 10 indexé avec succès.
Article 11 indexé avec succès.
Article 12 indexé avec succès.
Article 13 indexé avec succès.
Article 14 indexé avec succès.
Article 15 indexé avec succès.
Article 16 indexé avec succès.
Article 17 indexé avec succès.
Article 18 indexé avec succès.
Article 19 indexé avec succès.
Article 20 indexé avec succès.
Article 21 indexé avec succès.
Article 22 indexé avec succès.
Article 23 indexé avec succès.
Article 24 indexé avec succès.
Article 25 indexé avec succès.
Article 26 indexé avec succès.
Article 27 indexé avec succès.
Article 28 indexé avec succès.
Article 29 indexé avec succès.
Article 30 indexé avec succès.
Article 31 indexé avec succès.
Article 32 indexé 

indexation des questions-reponses posées par les utilisateurs

In [ ]:

# URL ngrok
ngrok_url = 'https://dc48-102-180-19-51.ngrok-free.app'


In [ ]:
import requests  # Pour faire des requêtes HTTP
import json  # Pour manipuler des objets JSON

# Fonction pour créer un nouvel index dans Elasticsearch pour les questions et réponses
def create_questions_index(ngrok_url):
    index_name = 'questions_reponses'
    index_body = {
        "mappings": {
            "properties": {
                "question": {"type": "text"},
                "response": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 768}  # Adapter selon la taille de tes embeddings
            }
        }
    }

    response = requests.put(f'{ngrok_url}/{index_name}', headers={"Content-Type": "application/json"}, data=json.dumps(index_body))

    if response.status_code == 200:
        print(f"L'index {index_name} a été créé avec succès.")
    else:
        print(f"Erreur lors de la création de l'index : {response.text}")

# Exécute cette fonction pour créer l'index dans Colab
create_questions_index(ngrok_url)


Erreur lors de la création de l'index : 400 - {"error":{"root_cause":[{"type":"resource_already_exists_exception","reason":"index [questions_reponses/v3ZcArKTTVaOPMkGH0c6Dw] already exists","index_uuid":"v3ZcArKTTVaOPMkGH0c6Dw","index":"questions_reponses"}],"type":"resource_already_exists_exception","reason":"index [questions_reponses/v3ZcArKTTVaOPMkGH0c6Dw] already exists","index_uuid":"v3ZcArKTTVaOPMkGH0c6Dw","index":"questions_reponses"},"status":400}


In [ ]:
# Fonction pour indexer une nouvelle question et réponse
def index_question_in_elasticsearch(question, response, embedding, ngrok_url):
    doc = {
        'question': question,
        'response': response,
        'embedding': embedding.tolist()  # Transformer en liste pour JSON
    }

    # Imprimer le document pour vérifier son contenu
    print("Document à indexer :", json.dumps(doc, indent=4))

    # Envoyer la requête POST pour indexer
    response = requests.post(f'{ngrok_url}/questions_reponses/_doc/', headers={"Content-Type": "application/json"}, data=json.dumps(doc))

    if response.status_code != 201:
        print(f"Erreur d'indexation pour la question : {response.status_code} - {response.text}")
    else:
        print("Question et réponse indexées avec succès.")


In [ ]:
import requests

# URL de Ngrok
ngrok_url = 'https://dc48-102-180-19-51.ngrok-free.app'

# Vérifier la liste des index
response = requests.get(f'{ngrok_url}/_cat/indices?v')

if response.status_code == 200:
    print("Liste des index disponibles :")
    print(response.text)
else:
    print(f"Erreur lors de la récupération des index : {response.status_code} - {response.text}")


Liste des index disponibles :
health status index                        uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .geoip_databases             Hq-OakysQLOHzfzrSh_JTw   1   0         38            0     36.4mb         36.4mb
yellow open   securite_v2                  _l2LE77NRdy5PN0yMD5thA   1   1          0            0       227b           227b
yellow open   police_judiciaire_embeddings RGGOENxwSwKrt7UvasIWAQ   1   1       1040            0     20.3mb         20.3mb
yellow open   droits_humains_v2            jKoLyC_aSwKMiVtOkY5fag   1   1          0            0       227b           227b
yellow open   police_nationale_v2          kiziwY6oTmmlrTJMHcX55g   1   1          0            0       227b           227b
yellow open   police_nationale_embeddings  hvoNkyKDT6Wx7mVJAjaBBA   1   1        911            0     17.5mb         17.5mb
yellow open   police_judiciaire_v2         FPysYgx8Q82D2euEqMqFlg   1   1          0            0     

In [ ]:
import numpy as np
import json
import requests

# URL ngrok
ngrok_url = 'https://dc48-102-180-19-51.ngrok-free.app'

# Fonction pour tester l'indexation d'une question/réponse avec un vecteur d'embeddings de taille 768
def index_test_question(ngrok_url):
    # Générer un embedding de 768 dimensions
    embedding = np.random.rand(768).tolist()  # Génère un vecteur d'exemple avec 768 dimensions

    doc = {
        'question': "Quelle est la capitale de la France?",
        'response': "La capitale de la France est Paris.",
        'embedding': embedding  # Embedding généré
    }

    # Imprimer le document pour vérification
    print("Document à indexer :", json.dumps(doc, indent=4))

    # Envoyer la requête POST pour indexer
    response = requests.post(f'{ngrok_url}/questions_reponses/_doc/',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(doc))

    if response.status_code == 201:
        print("Question et réponse indexées avec succès.")
    else:
        print(f"Erreur lors de l'indexation : {response.status_code} - {response.text}")

# Tester l'indexation manuellement dans Google Colab
index_test_question(ngrok_url)


Document à indexer : {
    "question": "Quelle est la capitale de la France?",
    "response": "La capitale de la France est Paris.",
    "embedding": [
        0.9436144489810114,
        0.42709563455431787,
        0.37964225613880453,
        0.9869590708164672,
        0.4011756143768709,
        0.1440032202297573,
        0.08308107603973691,
        0.9792658979978277,
        0.7864781231330801,
        0.6466233092451172,
        0.703937913454223,
        0.09208265074367272,
        0.9004194290844664,
        0.03362130295868948,
        0.8779037336909011,
        0.1408113473113587,
        0.7014934686571161,
        0.9778301033365554,
        0.8383428594386068,
        0.9071922173152547,
        0.9727404741482106,
        0.944656326071534,
        0.7207391675747413,
        0.3417481020004065,
        0.47995658339773817,
        0.4875137044956255,
        0.222698397966123,
        0.6498676754776331,
        0.3654466173811911,
        0.337939382534442,
     

In [ ]:
# Fonction pour vérifier les questions/réponses stockées dans Elasticsearch
def get_all_questions_responses(ngrok_url):
    search_query = {
        "query": {
            "match_all": {}  # Requête pour récupérer tous les documents
        }
    }

    response = requests.post(f'{ngrok_url}/questions_reponses/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))

    if response.status_code == 200:
        results = response.json()
        if results['hits']['total']['value'] > 0:
            return results['hits']['hits']  # Retourne les documents trouvés
        else:
            print("Aucun document trouvé dans Elasticsearch.")
    else:
        print(f"Erreur lors de la récupération des questions/réponses : {response.status_code} - {response.text}")

# Vérifier les questions/réponses indexées
get_all_questions_responses(ngrok_url)


[{'_index': 'questions_reponses',
  '_type': '_doc',
  '_id': '6XsnU5IBTaXCknhKIULE',
  '_score': 1.0,
  '_source': {'question': 'Quelle est la capitale de la France?',
   'response': 'La capitale de la France est Paris.',
   'embedding': [0.9436144489810114,
    0.42709563455431787,
    0.37964225613880453,
    0.9869590708164672,
    0.4011756143768709,
    0.1440032202297573,
    0.08308107603973691,
    0.9792658979978277,
    0.7864781231330801,
    0.6466233092451172,
    0.703937913454223,
    0.09208265074367272,
    0.9004194290844664,
    0.03362130295868948,
    0.8779037336909011,
    0.1408113473113587,
    0.7014934686571161,
    0.9778301033365554,
    0.8383428594386068,
    0.9071922173152547,
    0.9727404741482106,
    0.944656326071534,
    0.7207391675747413,
    0.3417481020004065,
    0.47995658339773817,
    0.4875137044956255,
    0.222698397966123,
    0.6498676754776331,
    0.3654466173811911,
    0.337939382534442,
    0.46737570842172294,
    0.32591477885

debut

In [ ]:
def classify_question(query_text):
    # Définir des mots-clés pour chaque institution
    droits_humains_keywords = ["handicap", "indigence", "droits humains", "protection des personnes handicapées", "inclusion sociale", "personnes vulnérables"]
    police_judiciaire_keywords = ["violence", "crimes", "délits", "prévention de la violence", "justice pénale", "enquête judiciaire", "procédure pénale"]
    police_nationale_keywords = ["académie de police", "formation policière", "forces de l'ordre", "admission stagiaires", "missions de police", "sécurité publique"]
    securite_keywords = ["gardiennage", "sociétés privées de sécurité", "sécurité nationale", "sécurité privée", "protection des biens", "responsabilité professionnelle"]

    # Vérifier quel ensemble de mots-clés correspond le mieux à la question
    if any(keyword in query_text.lower() for keyword in droits_humains_keywords):
        return "droits_humains_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_judiciaire_keywords):
        return "police_judiciaire_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_nationale_keywords):
        return "police_nationale_embeddings"
    elif any(keyword in query_text.lower() for keyword in securite_keywords):
        return "securite_embeddings"
    else:
        # Si aucun mot-clé ne correspond, on peut renvoyer une valeur par défaut
        return "general_embeddings"


verification de la classification

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 74.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**Première étape de classification**

In [ ]:
def classify_question(query_text):
    # Définir des mots-clés pour chaque institution
    droits_humains_keywords = ["handicap", "indigence", "droits humains", "protection des personnes handicapées", "inclusion sociale", "personnes vulnérables"]
    police_judiciaire_keywords = ["violence", "crimes", "délits", "prévention de la violence", "justice pénale", "enquête judiciaire", "procédure pénale"]
    police_nationale_keywords = ["académie de police", "formation policière", "forces de l'ordre", "admission stagiaires", "missions de police", "sécurité publique","police national"]
    securite_keywords = ["gardiennage", "sociétés privées de sécurité", "sécurité nationale", "sécurité privée", "protection des biens", "responsabilité professionnelle"]

    institutions = []

    # Vérifier les mots-clés correspondants
    if any(keyword in query_text.lower() for keyword in droits_humains_keywords):
        institutions.append("droits_humains_embeddings")
    if any(keyword in query_text.lower() for keyword in police_judiciaire_keywords):
        institutions.append("police_judiciaire_embeddings")
    if any(keyword in query_text.lower() for keyword in police_nationale_keywords):
        institutions.append("police_nationale_embeddings")
    if any(keyword in query_text.lower() for keyword in securite_keywords):
        institutions.append("securite_embeddings")

    # Si aucun mot-clé ne correspond, renvoyer une valeur par défaut
    if not institutions:
        institutions.append("general_embeddings")

    return institutions


In [ ]:
# Cas de test avec plusieurs institutions
test_cases = {
    "Quels sont les droits humains des personnes handicapées et comment la police judiciaire gère-t-elle les crimes contre eux ?": ["droits_humains_embeddings", "police_judiciaire_embeddings"],
    "Comment la sécurité privée protège-t-elle les biens tout en respectant les droits humains ?": ["droits_humains_embeddings", "securite_embeddings"],
    "Quel est le rôle de la police nationale dans l'admission des stagiaires ?": [ "police_nationale_embeddings"],
    "Cette question concerne la sécurité et les crimes, mais elle n'est pas ciblée.": ["police_judiciaire_embeddings", "securite_embeddings"],
    "Ceci est une question générale sans mots-clés pertinents.": ["general_embeddings"],
    "Quels droits les personnes vulnérables ont-elles en matière de protection, et comment la police judiciaire intervient-elle ?": ["droits_humains_embeddings", "police_judiciaire_embeddings"],
    "Comment la police nationale assure-t-elle la sécurité publique tout en respectant les droits des citoyens ?": ["droits_humains_embeddings", "police_nationale_embeddings"],
    "Quels sont les enjeux de la sécurité privée dans la protection des biens publics et privés ?": ["securite_embeddings"],
    "Les procédures judiciaires concernant les crimes de violence doivent-elles tenir compte des droits humains ?": ["droits_humains_embeddings", "police_judiciaire_embeddings"],
    "Quelle est la responsabilité des sociétés de sécurité privée en matière de sécurité nationale ?": ["securite_embeddings"],
    "Quels types de formation sont offerts à l'académie de police pour gérer la violence ?": ["police_nationale_embeddings", "police_judiciaire_embeddings"],
    "Comment les droits humains sont-ils intégrés dans les procédures de la police judiciaire ?": ["droits_humains_embeddings", "police_judiciaire_embeddings"],
    "Quels sont les défis rencontrés par la police nationale dans la gestion des stagiaires ?": ["police_nationale_embeddings"],
    "Comment la sécurité privée et la police nationale collaborent-elles pour assurer la sécurité publique ?": ["securite_embeddings", "police_nationale_embeddings"],
    "Quels droits ont les victimes de crimes, et comment la police judiciaire les protège-t-elle ?": ["droits_humains_embeddings", "police_judiciaire_embeddings"]
}

# Exécuter les tests
for question, expected_output in test_cases.items():
    result = classify_question(question)
    assert result == expected_output, f"Échec pour '{question}' : attendu {expected_output}, obtenu {result}"
    print(f"Test réussi pour: '{question}' -> {result}")


AssertionError: Échec pour 'Quels sont les droits humains des personnes handicapées et comment la police judiciaire gère-t-elle les crimes contre eux ?' : attendu ['droits_humains_embeddings', 'police_judiciaire_embeddings'], obtenu ['droits_humains_embeddings', 'police_judiciaire_embeddings', 'police_nationale_embeddings']

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

def evaluate_classification(test_cases):
    y_true = []
    y_pred = []

    for question, expected_output in test_cases.items():
        result = classify_question(question)

        # Ajouter les résultats aux listes
        y_true.append(expected_output)
        y_pred.append(result)

    # Utiliser MultiLabelBinarizer pour convertir les listes de listes en format binaire
    mlb = MultiLabelBinarizer()
    y_true_bin = mlb.fit_transform(y_true)
    y_pred_bin = mlb.transform(y_pred)

    # Calculer les métriques
    precision = precision_score(y_true_bin, y_pred_bin, average='weighted', zero_division=0)
    recall = recall_score(y_true_bin, y_pred_bin, average='weighted', zero_division=0)
    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted', zero_division=0)

    return precision, recall, f1


In [ ]:
# Exécuter l'évaluation
precision, recall, f1 = evaluate_classification(test_cases)
print(f"Précision: {precision:.2f}, Rappel: {recall:.2f}, F1-Score: {f1:.2f}")


Précision: 0.56, Rappel: 1.00, F1-Score: 0.71


**Seconde classification avec lemmatisation et synonyme**

fin de la verification

In [ ]:
def search_combined_in_elasticsearch(query_text, ngrok_url, index_name, weight_text=0.7, weight_vector=0.3):
    # Générer l'embedding pour la requête
    tokens = tokenizer(query_text, truncation=True, return_tensors='tf')['input_ids']
    inputs = {'input_ids': tokens}
    outputs = model(inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].numpy().flatten()

    # Construire la requête Elasticsearch
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "text": query_text  # Recherche textuelle classique
                        }
                    },
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}  # Appliquer la recherche vectorielle sur tous les documents
                            },
                            "script": {
                                "source": f"cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {
                                    "query_vector": query_embedding.tolist()
                                }
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }
    }

    # Exécuter la requête
    response = requests.post(f'{ngrok_url}/{index_name}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))
    if response.status_code == 200:
        results = response.json()
        print(f"Résultats de la recherche combinée pour '{query_text}':")
        for hit in results['hits']['hits']:
            print(f"Article ID: {hit['_source']['article_id']}, Score: {hit['_score']}, Text: {hit['_source']['text']}")
    else:
        print(f"Erreur lors de la recherche: {response.text}")


In [ ]:
query_text = "personnes handicapées"
index_name = "droits_humains_embeddings"
# Lancer la recherche combinée
search_combined_in_elasticsearch(query_text, ngrok_url, index_name)

Erreur lors de la recherche: {"error":{"root_cause":[{"type":"script_exception","reason":"runtime error","script_stack":["org.elasticsearch.xpack.vectors.query.ScoreScriptUtils$DenseVectorFunction.<init>(ScoreScriptUtils.java:59)","org.elasticsearch.xpack.vectors.query.ScoreScriptUtils$CosineSimilarity.<init>(ScoreScriptUtils.java:172)","score = cosineSimilarity(params.query_vector, 'embedding');\n                                    ","                                              ^---- HERE"],"script":" ...","lang":"painless","position":{"offset":90,"start":44,"end":140}}],"type":"search_phase_execution_exception","reason":"all shards failed","phase":"query","grouped":true,"failed_shards":[{"shard":0,"index":"droits_humains_embeddings","node":"gVWDesknQTahbxP72m8rUg","reason":{"type":"script_exception","reason":"runtime error","script_stack":["org.elasticsearch.xpack.vectors.query.ScoreScriptUtils$DenseVectorFunction.<init>(ScoreScriptUtils.java:59)","org.elasticsearch.xpack.vectors.q

In [ ]:
!pip install langchain streamlit transformers tensorflow


In [ ]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

# Charger le modèle GPT-2
def load_gpt2_model():
    model_name = 'gpt2'  # Vous pouvez choisir un autre modèle GPT si vous le souhaitez
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = TFGPT2LMHeadModel.from_pretrained(model_name)
    return model, tokenizer

model, tokenizer = load_gpt2_model()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Template de prompt pour combiner la question et les articles
prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template="Voici des articles pertinents : {context}. Réponds à la question suivante : {question}"
)

# Chaîne LLM avec LangChain
def create_llm_chain(model, tokenizer):
    return LLMChain(llm=model, prompt=prompt_template)

llm_chain = create_llm_chain(model, tokenizer)


<ipython-input-22-e78a87ee384a>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(llm=model, prompt=prompt_template)


ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.models.gpt2...bject at 0x7a5fba66bc70>, input_type=TFGPT2LMHeadModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=<transformers.models.gpt2...bject at 0x7a5fba66bc70>, input_type=TFGPT2LMHeadModel]
    For further information visit https://errors.pydantic.dev/2.9/v/is_instance_of

In [ ]:
def process_pipeline(query_text, ngrok_url):
    # Étape 1 : Classer la question pour déterminer l'institution
    index_name = classify_question(query_text)

    # Étape 2 : Recherche dans Elasticsearch en utilisant l'index de l'institution classée
    search_results = search_combined_in_elasticsearch(query_text, ngrok_url, index_name)

    # Étape 3 : Extraire les textes des articles récupérés
    context = "\n".join([hit['_source']['text'] for hit in search_results['hits']['hits']])

    # Étape 4 : Utiliser GPT-2 via LangChain pour générer la réponse
    inputs = tokenizer.encode(f"Articles : {context}\nQuestion : {query_text}\nRéponse :", return_tensors="tf")
    output = model.generate(inputs, max_length=150, num_return_sequences=1)

    # Décoder et retourner la réponse générée
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response


fin

**Recherche d'informations**

Recherche textuelle juste test

In [ ]:
def search_full_text(ngrok_url, index_name, query_text):
    search_query = {
        "query": {
            "match": {
                "text": query_text  # Rechercher les articles correspondant à ce texte
            }
        }
    }

    # Envoyer la requête à Elasticsearch
    response = requests.post(f'{ngrok_url}/{index_name}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))
    if response.status_code == 200:
        results = response.json()
        return results['hits']['hits']  # Retourne les documents trouvés
    else:
        print(f"Erreur lors de la recherche : {response.text}")
        return None

# Exemple d'appel de la fonction pour une recherche de texte simple
full_text_results = search_full_text(ngrok_url, 'droits_humains_embeddings', "personnes handicapées")
if full_text_results:
    for result in full_text_results:
        print(f"Document trouvé : {result['_source']['text']}")


Document trouvé : Article Article 6 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées susceptibles de bénéficier d une bourse de formation professionnelle est reculée de deux 02 ans en cas de besoin 
Document trouvé : Article Article 6 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées susceptibles de bénéficier d une bourse de formation professionnelle est reculée de deux 02 ans en cas de besoin 
Document trouvé : Article Article 10 : Loi DECRET N2012-829 : un quota d emploi de 10 dans la fonction publique et dans les établissements publics de l etat est réservé aux personnes handicapées selon leur qualification et conformément aux textes régissant les différents emplois 
Document trouvé : Article Article 10 : Loi DECRET N2012-829 : un quota d emploi de 10 dans la fonction publique et dans les établissements publics de l etat est réservé aux personnes handicapées selon leur qualification et conformément aux textes régissant les différents emplois 
Docu

In [ ]:
!pip install google-generativeai


In [ ]:
import requests
import json

# Fonction pour exécuter une recherche en texte intégral dans Elasticsearch
def search_full_text(ngrok_url, index_name, query_text):
    search_query = {
        "query": {
            "match": {
                "text": query_text  # Rechercher les articles correspondant à ce texte
            }
        }
    }

    # Envoyer la requête à Elasticsearch
    response = requests.post(f'{ngrok_url}/{index_name}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))

    # Gestion des réponses et des erreurs
    if response.status_code == 200:
        results = response.json()
        return results['hits']['hits']  # Retourne les documents trouvés
    else:
        print(f"Erreur lors de la recherche : {response.text}")
        return None

# Utiliser la fonction de classification pour déterminer l'index approprié
def classify_question(query_text):
    droits_humains_keywords = ["handicap", "indigence", "droits humains", "protection des personnes handicapées", "inclusion sociale", "personnes vulnérables"]
    police_judiciaire_keywords = ["violence", "crimes", "délits", "prévention de la violence", "justice pénale", "enquête judiciaire", "procédure pénale"]
    police_nationale_keywords = ["académie de police", "formation policière", "forces de l'ordre", "admission stagiaires", "missions de police", "sécurité publique"]
    securite_keywords = ["gardiennage", "sociétés privées de sécurité", "sécurité nationale", "sécurité privée", "protection des biens", "responsabilité professionnelle"]

    if any(keyword in query_text.lower() for keyword in droits_humains_keywords):
        return "droits_humains_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_judiciaire_keywords):
        return "police_judiciaire_embeddings"
    elif any(keyword in query_text.lower() for keyword in police_nationale_keywords):
        return "police_nationale_embeddings"
    elif any(keyword in query_text.lower() for keyword in securite_keywords):
        return "securite_embeddings"
    else:
        return "general_embeddings"

# Exemple d'appel de la fonction pour une recherche en texte intégral
query_text = "personnes handicapées"
index_name = classify_question(query_text)  # Classifier la question pour déterminer l'index
full_text_results = search_full_text(ngrok_url, index_name, query_text)

# Affichage des résultats trouvés
if full_text_results:
    print(f"Documents trouvés dans l'index {index_name} :")
    for result in full_text_results:
        print(f"Document ID: {result['_id']}, Texte : {result['_source']['text']}")
else:
    print("Aucun résultat trouvé.")


Documents trouvés dans l'index droits_humains_embeddings :
Document ID: l6weD5IBhlp8jtEH7UsR, Texte : Article Article 6 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées susceptibles de bénéficier d une bourse de formation professionnelle est reculée de deux 02 ans en cas de besoin 
Document ID: qqwxD5IBhlp8jtEHhkxU, Texte : Article Article 6 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées susceptibles de bénéficier d une bourse de formation professionnelle est reculée de deux 02 ans en cas de besoin 
Document ID: m6weD5IBhlp8jtEH90tj, Texte : Article Article 10 : Loi DECRET N2012-829 : un quota d emploi de 10 dans la fonction publique et dans les établissements publics de l etat est réservé aux personnes handicapées selon leur qualification et conformément aux textes régissant les différents emplois 
Document ID: rqwxD5IBhlp8jtEHj0xA, Texte : Article Article 10 : Loi DECRET N2012-829 : un quota d emploi de 10 dans la fonction publique et dans les ét

**Amélioration de la recherche pour combiner à la fois la recherche textuelle et vectorielle**

Creer un index avec champ embedding de type dense_vector pour effectuer la similarité cosinus ultérieurement

In [ ]:
# Fonction pour découper un texte en morceaux avec une fenêtre glissante
def sliding_window_tokenize(text, max_len=512, stride=256):
    tokens = tokenizer(text, truncation=False, return_tensors='tf')['input_ids'][0]
    token_chunks = []

    for i in range(0, len(tokens), stride):
        chunk = tokens[i:i + max_len]
        token_chunks.append(chunk)

        if len(chunk) < max_len:
            break  # Si le morceau est plus petit que la taille maximale, on arrête

    return token_chunks


In [ ]:
# Fonction pour générer des embeddings avec la fenêtre glissante
def generate_embeddings_with_sliding_window(text_list):
    all_embeddings = []

    for text in text_list:
        token_chunks = sliding_window_tokenize(text)
        chunk_embeddings = []

        for chunk in token_chunks:
            inputs = {'input_ids': tf.expand_dims(chunk, 0)}  # Ajouter une dimension batch
            outputs = model(inputs)
            embedding = outputs.last_hidden_state[:, 0, :]  # Extraire les embeddings [CLS]
            chunk_embeddings.append(embedding.numpy())

        # Moyenne des embeddings pour l'article complet
        avg_embedding = np.mean(chunk_embeddings, axis=0)
        # Aplatir l'embedding pour qu'il soit 1D
        flat_embedding = avg_embedding.flatten()
        all_embeddings.append(flat_embedding)

    return np.array(all_embeddings)


In [ ]:
# Fonction pour créer un index avec le champ embedding de type dense_vector
def create_elasticsearch_index(index_name, ngrok_url):
    index_mapping = {
        "mappings": {
            "properties": {
                "article_id": {"type": "keyword"},
                "text": {"type": "text"},
                "embedding": {"type": "dense_vector", "dims": 768}  # Embeddings de DistilBERT ont 768 dimensions
            }
        }
    }

    response = requests.put(f'{ngrok_url}/{index_name}',
                            headers={"Content-Type": "application/json"},
                            data=json.dumps(index_mapping))

    if response.status_code == 200:
        print(f"Index {index_name} créé avec succès.")
    else:
        print(f"Erreur lors de la création de l'index {index_name}: {response.text}")

# Créer les index avant d'indexer les documents
create_elasticsearch_index("droits_humains_embeddings", ngrok_url)
create_elasticsearch_index("police_judiciaire_embeddings", ngrok_url)
create_elasticsearch_index("police_nationale_embeddings", ngrok_url)
create_elasticsearch_index("securite_embeddings", ngrok_url)


Index droits_humains_embeddings créé avec succès.
Index police_judiciaire_embeddings créé avec succès.
Index police_nationale_embeddings créé avec succès.
Index securite_embeddings créé avec succès.


In [ ]:
# Fonction pour indexer les embeddings dans ElasticSearch
def index_embeddings_in_elasticsearch(articles, embeddings, index_name, ngrok_url):
    for i, (article, embedding) in enumerate(zip(articles, embeddings)):
        doc = {
            'article_id': f'article_{i}',
            'text': article,
            'embedding': embedding.tolist()  # Transformer en liste pour JSON
        }
        response = requests.post(f'{ngrok_url}/{index_name}/_doc/',
                                 headers={"Content-Type": "application/json"},
                                 data=json.dumps(doc))
        if response.status_code != 201:
            print(f"Erreur d'indexation pour l'article {i}: {response.text}")
        else:
            print(f"Article {i} indexé avec succès.")


In [ ]:
# Fonction pour traiter les données d'une institution
def process_institution(df, institution_name, ngrok_url):
    articles = []

    for index, row in df.iterrows():
        # Formater le texte de l'article avec le numéro de l'article et la loi
        formatted_text = f"Article {row['Num_Article']} Loi {row['loi']} : {row['Article']}"
        articles.append(formatted_text)

    # Générer les embeddings pour chaque article
    embeddings = generate_embeddings_with_sliding_window(articles)

    # Indexer les articles formatés et leurs embeddings dans Elasticsearch
    index_embeddings_in_elasticsearch(articles, embeddings, f"{institution_name.lower()}_embeddings", ngrok_url)


In [ ]:
# Traiter les données pour chaque institution
process_institution(Droits_humains, 'Droits_humains', ngrok_url)
process_institution(Police_Judiciaire, 'Police_Judiciaire', ngrok_url)
process_institution(Police_Nationale, 'Police_Nationale', ngrok_url)
process_institution(Securite, 'Securite', ngrok_url)


Article 0 indexé avec succès.
Article 1 indexé avec succès.
Article 2 indexé avec succès.
Article 3 indexé avec succès.
Article 4 indexé avec succès.
Article 5 indexé avec succès.
Article 6 indexé avec succès.
Article 7 indexé avec succès.
Article 8 indexé avec succès.
Article 9 indexé avec succès.
Article 10 indexé avec succès.
Article 11 indexé avec succès.
Article 12 indexé avec succès.
Article 13 indexé avec succès.
Article 14 indexé avec succès.
Article 15 indexé avec succès.
Article 16 indexé avec succès.
Article 17 indexé avec succès.
Article 18 indexé avec succès.
Article 19 indexé avec succès.
Article 20 indexé avec succès.
Article 21 indexé avec succès.
Article 22 indexé avec succès.
Article 23 indexé avec succès.
Article 24 indexé avec succès.
Article 25 indexé avec succès.
Article 26 indexé avec succès.
Article 27 indexé avec succès.
Article 28 indexé avec succès.
Article 29 indexé avec succès.
Article 30 indexé avec succès.
Article 31 indexé avec succès.
Article 32 indexé 

In [ ]:
def search_combined_in_elasticsearch(query_text, ngrok_url, index_name, weight_text=0.7, weight_vector=0.3):
    # Générer l'embedding pour la requête
    tokens = tokenizer(query_text, truncation=True, return_tensors='tf')['input_ids']
    inputs = {'input_ids': tokens}
    outputs = model(inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].numpy().flatten()

    # Construire la requête Elasticsearch
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "should": [
                    {
                        "match": {
                            "text": query_text  # Recherche textuelle classique
                        }
                    },
                    {
                        "script_score": {
                            "query": {
                                "match_all": {}  # Appliquer la recherche vectorielle sur tous les documents
                            },
                            "script": {
                                "source": f"cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                                "params": {
                                    "query_vector": query_embedding.tolist()
                                }
                            }
                        }
                    }
                ],
                "minimum_should_match": 1
            }
        }
    }

    # Exécuter la requête
    response = requests.post(f'{ngrok_url}/{index_name}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))
    if response.status_code == 200:
        results = response.json()
        print(f"Résultats de la recherche combinée pour '{query_text}':")
        for hit in results['hits']['hits']:
            print(f"Article ID: {hit['_source']['article_id']}, Score: {hit['_score']}, Text: {hit['_source']['text']}")
    else:
        print(f"Erreur lors de la recherche: {response.text}")


In [ ]:
# Exemple de recherche combinée
search_combined_in_elasticsearch("personnes handicapées", ngrok_url, "droits_humains_embeddings")

Résultats de la recherche combinée pour 'personnes handicapées':
Article ID: article_5, Score: 8.674594, Text: Article Article 6 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées susceptibles de bénéficier d une bourse de formation professionnelle est reculée de deux 02 ans en cas de besoin 
Article ID: article_9, Score: 8.324791, Text: Article Article 10 : Loi DECRET N2012-829 : un quota d emploi de 10 dans la fonction publique et dans les établissements publics de l etat est réservé aux personnes handicapées selon leur qualification et conformément aux textes régissant les différents emplois 
Article ID: article_4, Score: 8.061365, Text: Article Article 5 : Loi DECRET N2012-829 : la limite d âge des personnes handicapées candidates aux examens et ou concours professionnels organisés par le s ministère s chargé s de la fonction publique et de l emploi est reculée de deux 02 ans en cas de besoin 
Article ID: article_7, Score: 7.486802, Text: Article Article 8 : Loi DEC

In [ ]:
def search_combined_in_multiple_indices(query_text, ngrok_url, indices, weight_text=0.7, weight_vector=0.3):
    # Tokeniser la requête pour générer l'embedding
    tokens = tokenizer(query_text, truncation=True, return_tensors='tf')['input_ids']
    inputs = {'input_ids': tokens}
    outputs = model(inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].numpy().flatten()

    # Construire la requête Elasticsearch
    search_query = {
        "size": 10,
        "query": {
            "script_score": {
                "query": {
                    "match": {
                        "text": query_text  # Recherche textuelle classique
                    }
                },
                "script": {
                    "source": f"cosineSimilarity(params.query_vector, 'embedding') + 1.0",  # Similarité cosinus
                    "params": {
                        "query_vector": query_embedding.tolist()
                    }
                }
            }
        }
    }

    # Construire la liste des index à rechercher
    indices_to_search = ",".join(indices)  # Combiner les index en une seule chaîne séparée par des virgules

    # Exécuter la requête
    response = requests.post(f'{ngrok_url}/{indices_to_search}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))

    if response.status_code == 200:
        results = response.json()
        print(f"Résultats de la recherche combinée pour '{query_text}':")
        for hit in results['hits']['hits']:
            print(f"Index: {hit['_index']}, Article ID: {hit['_source']['article_id']}, Score: {hit['_score']}, Text: {hit['_source']['text']}")
    else:
        print(f"Erreur lors de la recherche: {response.text}")

# Exemple d'utilisation avec plusieurs index
indices = ["droits_humains_embeddings", "police_judiciaire_embeddings", "police_nationale_embeddings", "securite_embeddings"]
search_combined_in_multiple_indices("personnes handicapées", ngrok_url, indices)


Résultats de la recherche combinée pour 'personnes handicapées':
Index: droits_humains_embeddings, Article ID: article_78, Score: 1.9168859, Text: Article Article 24 : Loi LOI N024-2016-AN : l’etat assure aux personnes âgées l’accès à l’information en rapport avec leurs besoins, à travers des supports d’information et de communication appropriés.
Index: droits_humains_embeddings, Article ID: article_239, Score: 1.9109277, Text: Article Article 1 : Loi LOI n 029-2008 : au sens de la présente loi, la traite des personnes désigne le recrutement, le transport, l’hébergement ou l’accueil de personnes, par le recours à la menace ou le recours à la force ou à d’autres formes de contrainte, par enlèvement, fraude, tromperie, abus d’autorité ou d’une situation de vulnérabilité, ou par l’offre ou l’acceptation de paiements ou d’avantages pour obtenir le consentement d’une personne ayant autorité sur une autre aux fins d’exploitation.l’exploitation comprend, notamment, l’exploitation de la prosti

**Fine-tuning de GPT2 de facon fédérée**

**Implementation Langchain et Streamlit**

In [ ]:
!pip install streamlit langchain pyngrok


In [ ]:
# Créer un tunnel ngrok pour exposer Streamlit
ngrok_tunnel = ngrok.connect(8501)
print(f"Streamlit est accessible publiquement via cette URL : {ngrok_tunnel.public_url}")

# Fonction pour exécuter une recherche multi-index dans Elasticsearch
def search_combined_in_multiple_indices(query_text, indices, ngrok_url):
    # Tokeniser la requête pour générer l'embedding
    tokens = tokenizer(query_text, truncation=True, return_tensors='tf')['input_ids']
    inputs = {'input_ids': tokens}
    outputs = model(inputs)
    query_embedding = outputs.last_hidden_state[:, 0, :].numpy().flatten()

    # Construire la requête Elasticsearch
    search_query = {
        "size": 10,
        "query": {
            "script_score": {
                "query": {
                    "match": {
                        "text": query_text  # Recherche textuelle classique
                    }
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",  # Similarité cosinus
                    "params": {
                        "query_vector": query_embedding.tolist()
                    }
                }
            }
        }
    }

    # Construire la liste des index à rechercher
    indices_to_search = ",".join(indices)

    # Exécuter la requête
    response = requests.post(f'{ngrok_url}/{indices_to_search}/_search',
                             headers={"Content-Type": "application/json"},
                             data=json.dumps(search_query))

    if response.status_code == 200:
        return response.json()
    else:
        st.error(f"Erreur lors de la recherche : {response.text}")
        return None

# Interface utilisateur avec Streamlit
st.title("Recherche d'informations juridiques")

# Entrée de la requête utilisateur
query = st.text_input("Posez votre question ici :")

# Liste des index à rechercher
indices = ["droits_humains_embeddings", "police_judiciaire_embeddings", "police_nationale_embeddings", "securite_embeddings"]

if st.button("Rechercher"):
    if query:
        # Exécuter la recherche multi-index
        results = search_combined_in_multiple_indices(query, indices, ngrok_url)

        if results:
            st.write("Résultats pertinents :")
            for hit in results['hits']['hits']:
                st.write(f"Index : {hit['_index']}, Article ID : {hit['_source']['article_id']}, Score : {hit['_score']}, Texte : {hit['_source']['text']}")

            # Préparer les résultats pour LangChain
            relevant_texts = [hit['_source']['text'] for hit in results['hits']['hits']]
            relevant_indices = [hit['_index'] for hit in results['hits']['hits']]

            # Template pour LangChain
            prompt_template = PromptTemplate(
                template="Voici les informations suivantes : {context}. Génère une réponse à la question : {question}",
                input_variables=["context", "question"]
            )

            # Modèle GPT-2
            llm = GPT2()

            # Chaîne LLM avec le modèle GPT-2 et le template
            chain = LLMChain(llm=llm, prompt=prompt_template)

            # Préparer le contexte pour GPT-2
            context = ". ".join(relevant_texts)

            # Générer une réponse avec GPT-2
            response = chain.run({"context": context, "question": query})

            st.write("Réponse générée :")
            st.write(response)
        else:
            st.write("Aucun résultat trouvé.")
    else:
        st.write("Veuillez poser une question.")

ERROR:pyngrok.process.ngrok:t=2024-09-20T01:48:12+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-20T01:48:12+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-09-20T01:48:12+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

**Implementation Gemini flash 1.5**

In [ ]:
!pip install google-generativeai

In [ ]:
!pip install langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, PDFMinerLoader, Docx2txtLoader, PyPDFLoader

load_dotenv()

True

In [ ]:

# Model Setup
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gen_config = {
    "temperature": 0.5,
    "max_output_tokens": 512
}

model = genai.GenerativeModel(
    model_name='gemini-1.5-flash',
    generation_config=gen_config
)

In [ ]:

# Function to generate a question and response based on the article content
def generate_question_response(article):
    prompt = f"""Contexte : {article}
Propose une question juridique pertinente à partir du contexte ci-dessus, puis génère une réponse qui est une reformulation du contexte."""
    response = model.generate_content(prompt)
    return response.text

# Example usage on a sample article
sample_article = Droits_humains['Article'].iloc[4]
question_response = generate_question_response(sample_article)
print(question_response)


## Question juridique :

**Est-ce que la possibilité de reculer la limite d'âge de deux ans pour les personnes handicapées candidates aux examens et concours professionnels organisés par le ministère chargé de la fonction publique et de l'emploi constitue une discrimination positive acceptable au regard du droit français ?**

## Réponse :

Le contexte mentionné indique que la limite d'âge pour les personnes handicapées candidates aux examens et concours professionnels organisés par le ministère chargé de la fonction publique et de l'emploi est reculée de deux ans en cas de besoin. Cette mesure vise à compenser le désavantage lié au handicap et à favoriser l'accès à l'emploi pour les personnes handicapées. 

